## NSE Segment FILES 

### NSE CODE without MOCK Trades

In [1]:
import os
import pandas as pd
from datetime import datetime

# === NSE Folder Paths ===
paths = {
    "FO": r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\FO",
    "EQ": r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\EQ",
    "CURRENCY": r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\CURRENCY",
}

# === Output folders ===
fo_output_folder = r"E:\DATA\2025-2026\GREEK TRADEBOOK"
eq_output_folder = r"E:\DATA\2025-2026\GREEK TRADEBOOK"
currency_output_folder = r"E:\DATA\2025-2026\GREEK TRADEBOOK"

# === Log file path ===
log_file_path = r"E:\DATA\2025-2026\MERGE_TRADEBOOK\MERGE_GREEK\merge_log.txt"

# === Headers ===
fo_headers = [
    "ExchangeTradeID", "Sample1", "Symbol", "SecurityType", "ExpiryDate",
    "StrikePrice", "OptionType", "SecurityName", "Sample2", "Sample3", "ManagerID",
    "Sample4", "Side", "Quantity", "Price", "Sample5", "ClientID", "MemberID",
    "Sample6", "ExchangeTradeTime", "ExchangeOrderTime", "ExchangeOrderNo",
    "ExchangeOrderStatus", "FinalExchangeOrderTime"
]

eq_headers = [
    "ExchangeTradeID", "Sample1", "Symbol", "SecurityType", "SecurityName",
    "Sample2", "Sample3", "Sample4", "ManagerID", "Sample5", "Side", "Quantity",
    "Price", "Sample6", "ClientID", "MemberID", "Sample7", "Sample8", "Sample9",
    "ExchangeTradeTime", "ExchangeOrderTime", "ExchangeOrderNo",
    "ExchangeOrderStatus", "FinalExchangeOrderTime"
]

currency_headers = [
    "ExchangeTradeID", "Sample1", "Symbol", "SecurityType", "SecurityName",
    "Sample2", "Quantity", "Price", "ExchangeTradeTime", "ExchangeOrderTime",
    "ExchangeOrderNo", "ExchangeOrderStatus"
]

def log(message):
    timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
    with open(log_file_path, "a", encoding="utf-8") as f:
        f.write(f"{timestamp} {message}\n")
    print(f"{timestamp} {message}")

def list_data_files(folder):
    if not os.path.exists(folder):
        log(f"⚠️ Folder not found: {folder}")
        return []
    return sorted([f for f in os.listdir(folder) if f.lower().endswith((".csv", ".txt"))])

def get_file_modified_date(filepath):
    try:
        return datetime.fromtimestamp(os.path.getmtime(filepath))
    except Exception:
        return None

def read_and_fix_file(filepath, headers, label):
    try:
        log(f"Reading {label} file: {filepath}")
        df = pd.read_csv(filepath, header=None, names=headers, encoding='utf-8', engine='python')
        df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

        try:
            df['ExchangeTradeTime'] = pd.to_datetime(df['ExchangeTradeTime'], format="%d %b %Y %H:%M:%S", errors='coerce')
        except Exception as e:
            log(f"Warning: Could not parse dates in {label} file {filepath}: {e}")

        date_min = df['ExchangeTradeTime'].min()

        if pd.isna(date_min):
            fallback_date = get_file_modified_date(filepath)
            if fallback_date:
                log(f"⚠️ Used file modified date as fallback for {label}: {fallback_date.strftime('%Y-%m-%d')}")
                date_min = fallback_date
            else:
                log(f"❌ Could not determine date from {label} file: {filepath}")
                return None, None

        if date_min.weekday() >= 5:
            log(f"🟡 Skipping {label} file as it's a Mock session (weekend): {filepath} -> {date_min.strftime('%A, %Y-%m-%d')}")
            return None, None

        log(f"✅ Parsed date from {label} file: {filepath} -> {date_min.strftime('%Y-%m-%d')}")
        return df, date_min

    except Exception as e:
        log(f"❌ Failed to read {label} file {filepath}: {e}")
        return None, None

def process_segment(segment_name, paths):
    log(f"--- Processing Segment: {segment_name} ---")

    fo_files = list_data_files(paths["FO"])
    eq_files = list_data_files(paths["EQ"])
    currency_files = list_data_files(paths["CURRENCY"])

    valid_currency_files = [f for f in currency_files if os.path.getsize(os.path.join(paths["CURRENCY"], f)) > 0]
    valid_eq_files = [f for f in eq_files if os.path.getsize(os.path.join(paths["EQ"], f)) > 0]

    processed_dates = set()

    for fo_file in fo_files:
        fo_path = os.path.join(paths["FO"], fo_file)
        if os.path.getsize(fo_path) == 0:
            log(f"🛑 Skipping empty FO file: {fo_file}")
            continue

        df_fo, date_fo = read_and_fix_file(fo_path, fo_headers, "FO")
        if date_fo is None:
            continue

        date_str = date_fo.strftime("%d-%m-%Y")
        if date_str in processed_dates:
            continue

        file_date_str = date_fo.strftime("%d%m%Y")

        # === Output FO file ===
        fo_out_path = os.path.join(fo_output_folder, f"FO_NSE_{file_date_str}.csv")
        df_fo.to_csv(fo_out_path, index=False, encoding='utf-8')
        log(f"✅ FO output saved: {fo_out_path}")

        # === EQ file processing (match by ExchangeTradeTime or fallback to file modified date) ===
        df_eq = None
        for eq_file in valid_eq_files:
            eq_path = os.path.join(paths["EQ"], eq_file)
            df_eq_temp, date_eq = read_and_fix_file(eq_path, eq_headers, "EQ")
            if df_eq_temp is None:
                continue

            if date_eq.strftime("%d-%m-%Y") == date_str:
                df_eq = df_eq_temp
                break

            file_mod_date = get_file_modified_date(eq_path)
            if file_mod_date and file_mod_date.strftime("%d-%m-%Y") == date_str:
                log(f"ℹ️ EQ fallback matched by file modified date: {eq_file}")
                df_eq = df_eq_temp
                break

        if df_eq is not None:
            eq_out_path = os.path.join(eq_output_folder, f"EQ_NSE_{file_date_str}.csv")
            df_eq.to_csv(eq_out_path, index=False, encoding='utf-8')
            log(f"✅ EQ output saved: {eq_out_path}")
        else:
            log(f"⚠️ No valid EQ file found for {date_str}. EQ skipped.")

        # === Currency file processing (same fallback logic) ===
        df_cur = None
        for cur_file in valid_currency_files:
            cur_path = os.path.join(paths["CURRENCY"], cur_file)
            df_cur_temp, date_cur = read_and_fix_file(cur_path, currency_headers, "CURRENCY")
            if df_cur_temp is None:
                continue

            if date_cur.strftime("%d-%m-%Y") == date_str:
                df_cur = df_cur_temp
                break

            file_mod_date = get_file_modified_date(cur_path)
            if file_mod_date and file_mod_date.strftime("%d-%m-%Y") == date_str:
                log(f"ℹ️ Currency fallback matched by file modified date: {cur_file}")
                df_cur = df_cur_temp
                break

        if df_cur is not None:
            cu_out_path = os.path.join(currency_output_folder, f"CU_NSE_{file_date_str}.csv")
            df_cur.to_csv(cu_out_path, index=False, encoding='utf-8')
            log(f"✅ Currency output saved: {cu_out_path}")
        else:
            log(f"⚠️ No valid Currency file found for {date_str}. Currency skipped.")

        processed_dates.add(date_str)
        log(f"🟢 Finished processing for trade date: {date_str}")

    log(f"--- Completed processing for segment: {segment_name} ---\n")

# === RUN NSE ===
log("======= NSE GREEK AUTO BACKUP PROCESS STARTED =======")
process_segment("NSE", paths)
log("======= NSE PROCESS COMPLETED =======")


[2025-06-17 17:32:41] ======= NSE GREEK AUTO BACKUP PROCESS STARTED =======
[2025-06-17 17:32:41] --- Processing Segment: NSE ---
[2025-06-17 17:32:48] Reading FO file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\FO\0116AUTOTRD.txt
[2025-06-17 17:32:48] ✅ Parsed date from FO file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\FO\0116AUTOTRD.txt -> 2025-01-16
[2025-06-17 17:32:48] ✅ FO output saved: E:\DATA\2025-2026\GREEK TRADEBOOK\FO_NSE_16012025.csv
[2025-06-17 17:32:48] Reading EQ file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\EQ\0117AUTOTRD.txt
[2025-06-17 17:32:49] ✅ Parsed date from EQ file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\EQ\0117AUTOTRD.txt -> 2025-01-17
[2025-06-17 17:32:49] Reading EQ file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\EQ\0118AUTOTRD.txt
[2025-06-17 17:32:49] ✅ Parsed date from EQ file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\EQ\0118AUTOTRD.txt -> 2025-01-17
[2025-06-17 17:32:49] Readin

### NSE CODE for Single Date

In [1]:
import os
import pandas as pd
from datetime import datetime

def log(message):
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {message}")

def is_weekend(date_obj):
    return date_obj.weekday() >= 5  # 5 = Saturday, 6 = Sunday

def parse_date_from_filename(filename):
    try:
        date_part = filename[:4]  # MMDD
        date_obj = datetime.strptime(date_part + '2025', "%m%d%Y")
        return date_obj
    except Exception:
        return None

def read_and_fix_file(path, headers, file_type, silent=False):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        if not silent:
            log(f"⚠️ Skipping zero size or missing {file_type} file: {os.path.basename(path)}")
        return None, None
    try:
        df = pd.read_csv(path, header=None, names=headers)
        file_date = parse_date_from_filename(os.path.basename(path))
        if not silent:
            log(f"📂 Reading {file_type} file: {os.path.basename(path)}")
            if file_date:
                log(f"✅ Parsed date from {file_type} file: {file_date.strftime('%Y-%m-%d')}")
        return df, file_date
    except Exception as e:
        if not silent:
            log(f"❌ Error reading {file_type} file {os.path.basename(path)}: {e}")
        return None, None

def process_segment(segment, paths, output_folder, target_date):
    log(f"--- Processing Segment: {segment} ---")

    target_date_obj = datetime.strptime(target_date, "%Y-%m-%d")

    # Skip entire processing if the target date is weekend
    if is_weekend(target_date_obj):
        log(f"🟡 Skipping all files processing for weekend date: {target_date}")
        log(f"--- Completed processing for segment: {segment} ---")
        return

    fo_files = sorted(os.listdir(paths.get("FO", ""))) if os.path.exists(paths.get("FO", "")) else []
    eq_files = sorted(os.listdir(paths.get("EQ", ""))) if os.path.exists(paths.get("EQ", "")) else []
    currency_files = sorted(os.listdir(paths.get("CURRENCY", ""))) if os.path.exists(paths.get("CURRENCY", "")) else []

    # === Headers ===
    fo_headers = [
        "ExchangeTradeID", "Sample1", "Symbol", "SecurityType", "ExpiryDate",
        "StrikePrice", "OptionType", "SecurityName", "Sample2", "Sample3", "ManagerID",
        "Sample4", "Side", "Quantity", "Price", "Sample5", "ClientID", "MemberID",
        "Sample6", "ExchangeTradeTime", "ExchangeOrderTime", "ExchangeOrderNo",
        "ExchangeOrderStatus", "FinalExchangeOrderTime"
    ]

    eq_headers = [
        "ExchangeTradeID", "Sample1", "Symbol", "SecurityType", "SecurityName",
        "Sample2", "Sample3", "Sample4", "ManagerID", "Sample5", "Side", "Quantity",
        "Price", "Sample6", "ClientID", "MemberID", "Sample7", "Sample8", "Sample9",
        "ExchangeTradeTime", "ExchangeOrderTime", "ExchangeOrderNo",
        "ExchangeOrderStatus", "FinalExchangeOrderTime"
    ]

    currency_headers = [
        "ExchangeTradeID", "Sample1", "Symbol", "SecurityType", "SecurityName",
        "Sample2", "Quantity", "Price", "ExchangeTradeTime", "ExchangeOrderTime",
        "ExchangeOrderNo", "ExchangeOrderStatus"
    ]

    file_date_str = target_date_obj.strftime("%d%m%Y")
    processed_any = False

    # Process FO
    processed_fo = False
    for fo_file in fo_files:
        fo_date = parse_date_from_filename(fo_file)
        if fo_date != target_date_obj:
            continue
        fo_path = os.path.join(paths["FO"], fo_file)
        if not os.path.exists(fo_path) or os.path.getsize(fo_path) == 0:
            log(f"⚠️ Skipping zero size or missing FO file: {fo_file}")
            continue
        df_fo, _ = read_and_fix_file(fo_path, fo_headers, "FO", silent=True)
        if df_fo is not None:
            log(f"✅ Processing FO file: {fo_file} -> {target_date}")
            fo_out = os.path.join(output_folder, f"FO_NSE_{file_date_str}.csv")
            df_fo.to_csv(fo_out, index=False, encoding='utf-8')
            log(f"✅ FO output saved: {fo_out}")
            processed_fo = True
            processed_any = True
            break
    if not processed_fo:
        log(f"⚠️ No FO file found for date {target_date}")

    # Process EQ
    processed_eq = False
    for eq_file in eq_files:
        eq_date = parse_date_from_filename(eq_file)
        if eq_date != target_date_obj:
            continue
        eq_path = os.path.join(paths["EQ"], eq_file)
        if not os.path.exists(eq_path) or os.path.getsize(eq_path) == 0:
            log(f"⚠️ Skipping zero size or missing EQ file: {eq_file}")
            continue
        log(f"✅ Found matching EQ file: {eq_file}")
        df_eq, _ = read_and_fix_file(eq_path, eq_headers, "EQ", silent=True)
        if df_eq is not None:
            eq_out = os.path.join(output_folder, f"EQ_NSE_{file_date_str}.csv")
            df_eq.to_csv(eq_out, index=False, encoding='utf-8')
            log(f"✅ EQ output saved: {eq_out}")
            processed_eq = True
            processed_any = True
            break
    if not processed_eq:
        log(f"⚠️ No EQ file found for date {target_date}")

    # Process Currency
    processed_currency = False
    for cur_file in currency_files:
        cur_date = parse_date_from_filename(cur_file)
        if cur_date != target_date_obj:
            continue
        cur_path = os.path.join(paths["CURRENCY"], cur_file)
        if not os.path.exists(cur_path) or os.path.getsize(cur_path) == 0:
            log(f"⚠️ Skipping zero size or missing Currency file: {cur_file}")
            continue
        df_cur, _ = read_and_fix_file(cur_path, currency_headers, "CURRENCY", silent=True)
        if df_cur is not None:
            cur_out = os.path.join(output_folder, f"CU_NSE_{file_date_str}.csv")
            df_cur.to_csv(cur_out, index=False, encoding='utf-8')
            log(f"✅ Currency output saved: {cur_out}")
            processed_currency = True
            processed_any = True
            break
    if not processed_currency:
        log(f"⚠️ No Currency file found for date {target_date}")

    if processed_any:
        log(f"✅ Processed date: {target_date}")
    else:
        log(f"⚠️ No files found for date {target_date}")

    log(f"--- Completed processing for segment: {segment} ---")


# ==== CONFIGURATION ====
paths = {
    "FO": r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\FO",
    "EQ": r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\EQ",
    "CURRENCY": r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\NSE\Currency"
}

output_folder = r"E:\DATA\2025-2026\GREEK TRADEBOOK"
target_date = "2025-02-01"

# ==== RUN ====
log("======= NSE GREEK AUTO BACKUP PROCESS STARTED =======")
process_segment("NSE", paths, output_folder, target_date)
log("======= NSE PROCESS COMPLETED =======")


[2025-07-02 14:44:45] ======= NSE GREEK AUTO BACKUP PROCESS STARTED =======
[2025-07-02 14:44:45] --- Processing Segment: NSE ---
[2025-07-02 14:44:45] 🟡 Skipping all files processing for weekend date: 2025-02-01
[2025-07-02 14:44:45] --- Completed processing for segment: NSE ---
[2025-07-02 14:44:45] ======= NSE PROCESS COMPLETED =======


## BSE Segment Files

### BSE CODE without MOCK Trades 

In [2]:
import os
import pandas as pd
from datetime import datetime

# === Folder Paths ===
fo_input_folder = r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\FO"
eq_input_folder = r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\EQ"
cu_input_folder = r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\CURRENCY"
output_folder = r"E:\DATA\2025-2026\GREEK TRADEBOOK"
log_file_path = r"E:\DATA\2025-2026\MERGE_TRADEBOOK\MERGE_GREEK\merge_log_BSE.txt"

# === Column Headers ===
fo_columns = [
    "Empty1", "Empty2", "Empty3", "Empty4", "ExchangeTradeID", "Symbol", "Side", "Quantity", "Price",
    "ManagerID", "Status", "Empty5", "TradeDate1", "TradeTime1", "TradeDate2", "TradeTime2", "ExchangeOrderNo",
    "SecurityType", "Empty6", "ClientCode", "Instruction"
]

eq_columns = [
    "ExchangeTradeID", "ManagerID", "SecurityCode", "Symbol", "Sample1", "Val1", "Val2", "Val3",
    "ExchangeTradeTime", "ExchangeTradeDate", "ClientCode", "ExchangeOrderNo", "PositionType", "Side",
    "OrderID", "Ownership", "ISIN", "Flag1", "Flag2", "TradeTime2", "Extra"
]

cu_columns = [
    "Col1", "Col2", "Col3", "Col4", "Col5", "Col6", "Col7", "Col8",
    "Col9", "Col10", "Col11", "Col12", "Col13", "Col14", "Col15",
    "Col16", "Col17", "Col18", "Col19", "Col20"
]

# === Logging ===
def log(message):
    timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
    with open(log_file_path, "a", encoding="utf-8") as f:
        f.write(f"{timestamp} {message}\n")
    print(f"{timestamp} {message}")

# === File Listing ===
def list_data_files(folder):
    if not os.path.exists(folder):
        log(f"⚠️ Folder not found: {folder}")
        return []
    return sorted([f for f in os.listdir(folder) if f.lower().endswith((".csv", ".txt"))])

# === File Reader with Weekend Skip ===
def read_and_fix_file(filepath, headers, label):
    try:
        if os.path.getsize(filepath) == 0:
            log(f"⚠️ Skipping empty {label} file: {filepath}")
            return None, None

        df = pd.read_csv(filepath, header=None, names=headers, sep="|", encoding='utf-8', engine='python')
        df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

        # Determine which column to use for date
        if label == "FO":
            date_column = "TradeDate1" if "TradeDate1" in df.columns else None
            date_format = "%d/%m/%Y"
        elif label == "EQ":
            date_column = "TradeDate1" if "TradeDate1" in df.columns else "Col10"
            date_format = "%Y/%m/%d"
        elif label == "CU":
            date_column = "Col10"
            date_format = "%Y/%m/%d"
        else:
            date_column = None

        if date_column and date_column in df.columns:
            df[date_column] = pd.to_datetime(df[date_column], format=date_format, errors='coerce')
            date_min = df[date_column].min()
        else:
            date_min = None

        if pd.isna(date_min):
            log(f"⚠️ Could not extract valid date from {label} file: {filepath}")
            return None, None
        elif date_min.weekday() >= 5:  # 5 = Saturday, 6 = Sunday
            log(f"🟡 Skipping {label} file as it's a Mock session (weekend): {filepath} -> {date_min.strftime('%A, %Y-%m-%d')}")
            return None, None
        else:
            log(f"✅ Parsed date from {label} file: {filepath} -> {date_min.strftime('%Y-%m-%d')}")

        return df, date_min

    except Exception as e:
        log(f"❌ Failed to read {label} file {filepath}: {e}")
        return None, None

# === Main Processing ===
def process_bse_data():
    log("======= BSE GREEK TRADEBOOK PROCESSING STARTED =======")

    fo_files = list_data_files(fo_input_folder)
    eq_files = list_data_files(eq_input_folder)
    cu_files = list_data_files(cu_input_folder)

    processed_dates = set()
    fo_data_by_date = {}

    # === Process FO files and collect valid trade dates ===
    for fo_file in fo_files:
        fo_path = os.path.join(fo_input_folder, fo_file)
        df_fo, date_fo = read_and_fix_file(fo_path, fo_columns, "FO")
        if df_fo is None or pd.isna(date_fo):
            continue

        date_str = date_fo.strftime("%d-%m-%Y")
        file_date_str = date_fo.strftime("%d%m%Y")
        processed_dates.add(date_str)
        fo_data_by_date[date_str] = df_fo

        output_fo_file = os.path.join(output_folder, f"FO_BSE_{file_date_str}.csv")
        df_fo.to_csv(output_fo_file, index=False, encoding='utf-8')
        log(f"✅ FO processed: {os.path.basename(output_fo_file)}")

    # === Process EQ files if their date matches a valid FO date ===
    for eq_file in eq_files:
        eq_path = os.path.join(eq_input_folder, eq_file)
        df_eq, date_eq = read_and_fix_file(eq_path, eq_columns, "EQ")
        if df_eq is None or pd.isna(date_eq):
            continue

        date_str = date_eq.strftime("%d-%m-%Y")
        if date_str not in processed_dates:
            continue

        file_date_str = date_eq.strftime("%d%m%Y")
        output_eq_file = os.path.join(output_folder, f"EQ_BSE_{file_date_str}.csv")
        df_eq.to_csv(output_eq_file, index=False, encoding='utf-8')
        log(f"✅ EQ processed: {os.path.basename(output_eq_file)}")

    # === Process CU files if their date matches a valid FO date ===
    for cu_file in cu_files:
        cu_path = os.path.join(cu_input_folder, cu_file)
        df_cu, date_cu = read_and_fix_file(cu_path, cu_columns, "CU")
        if df_cu is None or pd.isna(date_cu):
            continue

        date_str = date_cu.strftime("%d-%m-%Y")
        if date_str not in processed_dates:
            continue

        file_date_str = date_cu.strftime("%d%m%Y")
        output_cu_file = os.path.join(output_folder, f"CU_BSE_{file_date_str}.csv")
        df_cu.to_csv(output_cu_file, index=False, encoding='utf-8')
        log(f"✅ CU processed: {os.path.basename(output_cu_file)}")

    log("======= BSE GREEK TRADEBOOK PROCESSING COMPLETED =======")

# === Execute ===
if __name__ == "__main__":
    process_bse_data()


[2025-06-17 17:49:34] ======= BSE GREEK TRADEBOOK PROCESSING STARTED =======
[2025-06-17 17:49:35] ⚠️ Skipping empty FO file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\FO\0117AUTOTRD.txt
[2025-06-17 17:49:35] ⚠️ Skipping empty FO file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\FO\0118AUTOTRD.txt
[2025-06-17 17:49:35] ⚠️ Skipping empty FO file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\FO\0120AUTOTRD.txt
[2025-06-17 17:49:35] ⚠️ Skipping empty FO file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\FO\0121AUTOTRD.txt
[2025-06-17 17:49:35] ⚠️ Skipping empty FO file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\FO\0122AUTOTRD.txt
[2025-06-17 17:49:35] ⚠️ Skipping empty FO file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\FO\0123AUTOTRD.txt
[2025-06-17 17:49:35] ⚠️ Skipping empty FO file: \\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\FO\0124AUTOTRD.txt
[2025-06-17 17:49:35] ⚠️ Skipping empty FO file: \\172.16.5.33\greek_

### BSE CODE for Single Date

In [9]:
import os
import pandas as pd
from datetime import datetime

# === Configs ===
target_date = "2025-07-07"
target_date_obj = datetime.strptime(target_date, "%Y-%m-%d")
target_date_str_file = target_date_obj.strftime("%d%m%Y")
output_folder = r"E:\DATA\2025-2026\GREEK TRADEBOOK"
log_file_path = r"E:\DATA\2025-2026\MERGE_TRADEBOOK\merge_log.txt"

fo_input_folder = r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\FO"
eq_input_folder = r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\EQ"
cu_input_folder = r"\\172.16.5.33\greek_admin_backup\AutoOnlineBackup\BSE\CURRENCY"

fo_columns = [
    "Empty1", "Empty2", "Empty3", "Empty4", "ExchangeTradeID", "Symbol", "Side", "Quantity", "Price",
    "ManagerID", "Status", "Empty5", "TradeDate1", "TradeTime1", "TradeDate2", "TradeTime2", "ExchangeOrderNo",
    "SecurityType", "Empty6", "ClientCode", "Instruction"
]

eq_columns = [
    "ExchangeTradeID", "ManagerID", "SecurityCode", "Symbol", "Sample1", "Val1", "Val2", "Val3",
    "ExchangeTradeTime", "ExchangeTradeDate", "ClientCode", "ExchangeOrderNo", "PositionType", "Side",
    "OrderID", "Ownership", "ISIN", "Flag1", "Flag2", "TradeTime2", "Extra"
]

cu_columns = [
    "Col1", "Col2", "Col3", "Col4", "Col5", "Col6", "Col7", "Col8",
    "Col9", "Col10", "Col11", "Col12", "Col13", "Col14", "Col15",
    "Col16", "Col17", "Col18", "Col19", "Col20"
]

# === Logging Function ===
def log(msg):
    timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
    full_msg = f"{timestamp} {msg}"
    with open(log_file_path, "a", encoding="utf-8") as f:
        f.write(full_msg + "\n")
    print(full_msg)

# === Parse date from filename ===
def parse_date_from_filename(filename):
    # filename example: 0618AUTOTRD.txt, first 4 chars MMDD
    try:
        mmdd = filename[:4]
        date_obj = datetime.strptime(mmdd + "2025", "%m%d%Y")
        return date_obj
    except Exception:
        return None

# === Read file safely ===
def read_file(filepath, headers, label):
    try:
        if not os.path.exists(filepath) or os.path.getsize(filepath) == 0:
            log(f"⚠️ Skipping empty {label} file: {os.path.basename(filepath)}")
            return None
        df = pd.read_csv(filepath, header=None, names=headers, sep="|", encoding="utf-8", engine="python")
        # Strip whitespace from string columns
        df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
        return df
    except Exception as e:
        log(f"❌ Failed to read {label} file {os.path.basename(filepath)}: {e}")
        return None

# === Weekend check helper ===
def is_weekend(date_obj):
    return date_obj.weekday() >= 5  # Saturday=5, Sunday=6

# === Main Process ===
def process_bse_data():
    log("======= BSE GREEK AUTO BACKUP PROCESS STARTED =======")
    log("--- Processing Segment: BSE ---")

    # Skip entire processing if target date is weekend
    if is_weekend(target_date_obj):
        log(f"🟡 Skipping all files processing for weekend date: {target_date}")
        log("--- Completed processing for segment: BSE ---")
        log("======= BSE PROCESS COMPLETED =======")
        return

    fo_files = sorted([f for f in os.listdir(fo_input_folder) if f.lower().endswith((".csv", ".txt"))])
    eq_files = sorted([f for f in os.listdir(eq_input_folder) if f.lower().endswith((".csv", ".txt"))])
    cu_files = sorted([f for f in os.listdir(cu_input_folder) if f.lower().endswith((".csv", ".txt"))])

    processed_any = False
    cu_found = False

    # --- FO ---
    for file in fo_files:
        file_date = parse_date_from_filename(file)
        if file_date == target_date_obj:
            log(f"✅ Parsed date from FO file: {file} -> {file_date.strftime('%Y-%m-%d')}")
            path = os.path.join(fo_input_folder, file)
            df = read_file(path, fo_columns, "FO")
            if df is not None:
                out_path = os.path.join(output_folder, f"FO_BSE_{target_date_str_file}.csv")
                df.to_csv(out_path, index=False, encoding="utf-8")
                log(f"✅ FO output saved: {out_path}")
                processed_any = True
                break

    # --- EQ ---
    for file in eq_files:
        file_date = parse_date_from_filename(file)
        if file_date == target_date_obj:
            log(f"✅ Parsed date from EQ file: {file} -> {file_date.strftime('%Y-%m-%d')}")
            path = os.path.join(eq_input_folder, file)
            df = read_file(path, eq_columns, "EQ")
            if df is not None:
                out_path = os.path.join(output_folder, f"EQ_BSE_{target_date_str_file}.csv")
                df.to_csv(out_path, index=False, encoding="utf-8")
                log(f"✅ EQ output saved: {out_path}")
                processed_any = True
                break

    # --- CU ---
    for file in cu_files:
        file_date = parse_date_from_filename(file)
        if file_date == target_date_obj:
            path = os.path.join(cu_input_folder, file)
            df = read_file(path, cu_columns, "Currency")
            if df is not None:
                out_path = os.path.join(output_folder, f"CU_BSE_{target_date_str_file}.csv")
                df.to_csv(out_path, index=False, encoding="utf-8")
                log(f"✅ CU output saved: {out_path}")
                processed_any = True
                cu_found = True
                break

    if not cu_found:
        log(f"⚠️ No Currency file found for date {target_date}")

    if processed_any:
        log(f"✅ Processed date: {target_date}")
    else:
        log(f"⚠️ No files found for date {target_date}")

    log("--- Completed processing for segment: BSE ---")
    log("======= BSE PROCESS COMPLETED =======")

# === Run ===
if __name__ == "__main__":
    process_bse_data()


[2025-07-07 12:52:06] ======= BSE GREEK AUTO BACKUP PROCESS STARTED =======
[2025-07-07 12:52:06] --- Processing Segment: BSE ---
[2025-07-07 12:52:06] ✅ Parsed date from FO file: 0626AUTOTRD.txt -> 2025-06-26
[2025-07-07 12:52:06] ✅ FO output saved: E:\DATA\2025-2026\GREEK TRADEBOOK\FO_BSE_26062025.csv
[2025-07-07 12:52:06] ✅ Parsed date from EQ file: 0626AUTOTRD.txt -> 2025-06-26
[2025-07-07 12:52:06] ⚠️ Skipping empty EQ file: 0626AUTOTRD.txt
[2025-07-07 12:52:06] ⚠️ Skipping empty Currency file: 0626AUTOTRD.txt
[2025-07-07 12:52:06] ⚠️ No Currency file found for date 2025-06-26
[2025-07-07 12:52:06] ✅ Processed date: 2025-06-26
[2025-07-07 12:52:06] --- Completed processing for segment: BSE ---
[2025-07-07 12:52:06] ======= BSE PROCESS COMPLETED =======
